In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
url = 'http://insideairbnb.com/get-the-data/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Scraping
body = soup.find('body', class_='layout-module--content--2bfc1')
div = body.find('div', {"id": "___gatsby"})
file_urls = div.find_all('a')

links = [link.get('href') for link in file_urls]

listings_list = []
reviews_list = []
for link in links:
    if link != None:
        if "united-states" in link and "listings.csv.gz" in link:
            listings_list.append(link)

        if "united-states" in link and "reviews.csv.gz" in link:
            reviews_list.append(link)


In [ ]:
## Running vader on each comment
big_df = pd.DataFrame()
count = len(reviews_list)
for i,review in zip(range(count),reviews_list):

  reviews_df = pd.read_csv(review)
  reviews_df = reviews_df[['listing_id','comments']]
  reviews_df = reviews_df.rename(columns={'listing_id':'id'})
  reviews_df['comments'] = reviews_df['comments'].replace('[^a-zA-Z\s]', '', regex=True)


  ### vader
  reviews_df['sentiment_score'] = reviews_df['comments'].apply(lambda comments:sid.polarity_scores(str(comments))['compound'])
  big_df = big_df.append(reviews_df)
  print("Complete",i,"of",count, review)

Complete 0 of 1 http://data.insideairbnb.com/united-states/ny/albany/2023-10-01/data/reviews.csv.gz


<ipython-input-11-35205dc31321>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df = big_df.append(reviews_df)


In [ ]:
# Function to remove stop words
english_stop_words = stopwords.words('english')
def remove_stop_words(review):
    if isinstance(review, str):
      removed_stop_words = [word for word in review.split() if word.lower() not in english_stop_words]
      return removed_stop_words
    else:
      return

In [ ]:
# Function for getting frequency of each word
from collections import Counter
def word_counter(word_list):
  counts = Counter(word_list)
  return(counts)

In [ ]:
# Function to get 50 most common words
def top_word(word_counter):
  return(word_counter.most_common(50))

In [ ]:
# Seggragating comments on the basis of sentiment score
big_df.loc[big_df['sentiment_score'] < 0, 'type_of_comm'] = 'negative'
big_df.loc[big_df['sentiment_score'] >= 0, 'type_of_comm'] = 'positive'

In [ ]:
# Cleaning the comments column
big_df['comments'] = big_df['comments'].astype("string")
big_df['comments'] = big_df['comments'] .fillna(" ")

In [ ]:
big_df_comments = big_df.groupby(['id','type_of_comm'], as_index=False).agg({'comments': ' '.join})

In [ ]:
big_df_comments["word_list"] = big_df_comments['comments'].apply(remove_stop_words)

In [ ]:
big_df_comments["word_counter"] = big_df_comments['word_list'].apply(word_counter)

In [ ]:
big_df_comments["top_word"] = big_df_comments['word_counter'].apply(top_word)

In [ ]:
big_df_comments['top_word'] = big_df_comments['top_word'].astype("string")


In [ ]:
big_df_comments=big_df_comments[['id','top_word','type_of_comm']]

# Extracting the list from the string
big_df_comments['Column2'] = big_df_comments['top_word'].apply(lambda x: eval(x))

expanded = big_df_comments['Column2'].explode().apply(pd.Series)

result_df = (
    pd.concat([big_df_comments.drop('Column2', axis=1), expanded.rename(columns={0: 'Column3', 1: 'Column4'})], axis=1)
    .reset_index(drop=True)
)

<ipython-input-24-8996e47fef1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df_comments['Column2'] = big_df_comments['top_word'].apply(lambda x: eval(x))


In [ ]:
# Final dataframe with id of listing and the top positive and negative words
result_df = result_df[["id", "type_of_comm", "Column3", "Column4"]]